Exercise 6.1 Poisson's equation
Let $\alpha \in \mathbb{R}$. We are given the Poisson problem in $1 \mathrm{D}$ on the interval $\Omega=(0,1)$ :
$$
\begin{aligned}
-\alpha u^{\prime \prime}(x) &=f \quad \text { in } \Omega \\
u(0)=u(1) &=0
\end{aligned}
$$
with $\alpha=1$ and the right hand side $f=-a$ with $a>0$. The code of this example can be found on the cloud in fem1d_linear.cc.
Note: Please note that the above form is only correct when $\alpha$ is constant. The general formulation is
$$
-\frac{d}{d x}\left(\alpha u^{\prime}\right)
$$
which reduces to the above one, when $\alpha$ is constant.
(a) Run the code and observe the results using gnuplot, with $a=1$ and $h=0.1$.
Hint: Please work in the optimized compiling mode
(b) We play now with two parameters:
(i) Vary the discretization parameter $h$ and use other parameters. What do you observe?
(ii) Vary now the model parameter $\alpha$. What do you observe?
(iii) Choose now a different right hand side $f$. What do you observe?
(c) Check your solution by observing whether the maximum principle is fulfilled or not.
(d) We study in this final task the structure of the code. Go into the code and try to understand the different functions and methods that are implemented therein. Please have a specific look into the assemble_system method.

In [1]:
!cat fem1d_linear.cc

#include <iostream>  
#include <cmath>
#include "hdnum.hh" // hdnum header

/*
 * The class that contains all important parts of solving the Poisson problem.
 * 
 */
class Poisson{
public:
  Poisson(double h, double alpha,double a, int p = 1)
  :h(h),
   alpha(alpha),
   a(a),
   p(p)
  {}
  
  void run(); 
private:
  void make_grid();
  void setup_system();
  void assemble_system();
  void solve();
  void output_results() const;
  
  int    p;       //polynomial degree
  int    n_dofs;  //number of degrees of freedom 
  int    n_elems; //number of finite elements (intervals)
  
  double h;       //grid size
  double alpha;   //model parameter
  double a;       //rhs factor
  
  hdnum::Vector<double> rhs;      // right hand side (f)
  hdnum::Vector<double> solution; // solution vector (y)
  hdnum::Vector<double> grid;     // vector of grid points (x)
  
  hdnum::DenseMatrix<double> system_matrix; //system matrix   (A)
};

/*
 * This function calculates the subdivision of [0,1] 
 * base